In [1]:
import json
import base64
import qrcode
import requests
import datetime
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.message import Attachment, AttachmentDataJson, AttachmentDataLinks
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

In [2]:
secrets_resolver = SecretsResolverDemo()

In [3]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
            didcomm_services = [
                DIDCommService(
                    id = s.id,
                    service_endpoint = s.service_endpoint[0]["uri"] if "uri" in s.service_endpoint[0] else s.service_endpoint[0],
                    routing_keys = s.routing_keys,
                    accept = s.accept
                )
                for s in did_doc.service
                if isinstance(s, DIDCommServicePeerDID)
            ] if did_doc.service else []
        )

In [4]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, 
                    routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


In [3]:
import requests
peer_url ="https://mediator.rootsid.cloud/qr?_oobid=eaaadb5aa8da4081a7628eff199cfbfe"
data=requests.get(peer_url)
data.content

b'{"detail":"Not Found"}'

In [199]:
og_did_doc = json.loads(peer_did.resolve_peer_did(did_doc['service'][0]['serviceEndpoint']
))
og_did_doc


{'id': 'did:peer:2.Ez6LSmR579WxgKHvf9tU4fTqfY4RGh1XF5Dq7DfSR3uDEZH3Z.Vz6MkfyPuxGPEunGZ4hU15B7VvLKRMGd62NQHikV7TxXVX4FK.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0',
 'authentication': [{'id': 'did:peer:2.Ez6LSmR579WxgKHvf9tU4fTqfY4RGh1XF5Dq7DfSR3uDEZH3Z.Vz6MkfyPuxGPEunGZ4hU15B7VvLKRMGd62NQHikV7TxXVX4FK.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0#6MkfyPuxGPEunGZ4hU15B7VvLKRMGd62NQHikV7TxXVX4FK',
   'type': 'Ed25519VerificationKey2020',
   'controller': 'did:peer:2.Ez6LSmR579WxgKHvf9tU4fTqfY4RGh1XF5Dq7DfSR3uDEZH3Z.Vz6MkfyPuxGPEunGZ4hU15B7VvLKRMGd62NQHikV7TxXVX4FK.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0',
   'publicKeyMultibase': 'z6MkfyPuxGPEunGZ4hU15B7VvLKRMGd62NQHikV7TxXVX4FK'}],
 'keyAgreement': [{'id': 'did:peer:2.Ez6LSmR579WxgKHvf9tU4fTqfY4RGh1XF5Dq7DfSR3uDEZH3Z.Vz6MkfyPuxGPEunGZ4hU15B7V

In [200]:
#decode base64
import base64,json
peer_url ="https://mediator.rootsid.cloud/qr?_oobid=eaaadb5aa8da4081a7628eff199cfbfe"
data=requests.get(peer_url)
data
document=json.loads(base64.b64decode(data.url.split("=",1)[1]))

did_doc = json.loads(peer_did.resolve_peer_did(document['from']))
did_doc

{'id': 'did:peer:2.Ez6LSigBHnUyRnmH89fjjPfejFtm4RsBqXtFtZDLXPKRcqQ9y.Vz6MkkCvY6GmShnAmDV2Tpf5caGzgHNm5EE5XEyCqiZqgo3vh.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJkaWQ6cGVlcjoyLkV6NkxTa1h2Wm0xRHc4N3N4UVM0cndqU3NQbXUzb3AzQ1hkUTR1MUFnZHR3TXNaenguVno2TWtqSkFMTHhzV2dEOUM5eVQ1RU00M0Y2UEpYdlhkdGtDQ2NCdDVabVM2dkU3Ti5TZXlKcFpDSTZJbTVsZHkxcFpDSXNJblFpT2lKa2JTSXNJbk1pT2lKb2RIUndjem92TDIxbFpHbGhkRzl5TG5KdmIzUnphV1F1WTJ4dmRXUWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJhIjpbImRpZGNvbW0vdjIiXX0',
 'authentication': [{'id': 'did:peer:2.Ez6LSigBHnUyRnmH89fjjPfejFtm4RsBqXtFtZDLXPKRcqQ9y.Vz6MkkCvY6GmShnAmDV2Tpf5caGzgHNm5EE5XEyCqiZqgo3vh.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJkaWQ6cGVlcjoyLkV6NkxTa1h2Wm0xRHc4N3N4UVM0cndqU3NQbXUzb3AzQ1hkUTR1MUFnZHR3TXNaenguVno2TWtqSkFMTHhzV2dEOUM5eVQ1RU00M0Y2UEpYdlhkdGtDQ2NCdDVabVM2dkU3Ti5TZXlKcFpDSTZJbTVsZHkxcFpDSXNJblFpT2lKa2JTSXNJbk1pT2lKb2RIUndjem92TDIxbFpHbGhkRzl5TG5KdmIzUnphV1F1WTJ4dmRXUWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJhIjpbImRpZGNvbW0vdjIiXX0#6MkkCvY6GmShnAmDV2

### Get OOB get somehow Alice DID (not covered here) --> Copy did from Alice notebook

In [201]:
alice_did = "did:peer:2.Ez6LSmd2L4zT1XXpxGKWx42gcnHP3pvLMnSG8VocJEiYHLYkU.Vz6MkrmKEHEjjd1eZY4ETH55XqkuC5aBh1GLQMtHZJBc1cxov.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJkaWQ6cGVlcjoyLkV6NkxTcVV4dE5SZ2lOdzhjbWRBYWIyWVpIWDVLVTRrajFyaWdNWml6aTVMcWR1RW4uVno2TWtuUmhna2RDWjY5Tk5Ic1E4MUVreFhDZEt2a2Joc1pNSm1mazZUaVd0U3NzcS5TZXlKcFpDSTZJbTVsZHkxcFpDSXNJblFpT2lKa2JTSXNJbk1pT2lKb2RIUndjem92TDIxbFpHbGhkRzl5TG5KdmIzUnphV1F1WTJ4dmRXUWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJhIjpbImRpZGNvbW0vdjIiXX0"
alice_did = document['from']

### Get mediator DID from Alice DID and endpoint from mediator DID

### Alice acknowledge so the mediator can delete the message

In [202]:
alice_did_doc = json.loads(peer_did.resolve_peer_did(alice_did))
mediator_did = alice_did_doc["service"][0]["serviceEndpoint"][0]["uri"]
print(mediator_did)

did:peer:2.Ez6LSkXvZm1Dw87sxQS4rwjSsPmu3op3CXdQ4u1AgdtwMsZzx.Vz6MkjJALLxsWgD9C9yT5EM43F6PJXvXdtkCCcBt5ZmS6vE7N.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0


In [203]:
mediator_did_doc = json.loads(peer_did.resolve_peer_did(mediator_did))
mediator_endpoint = mediator_did_doc["service"][0]["serviceEndpoint"]
print(mediator_endpoint)

https://mediator.rootsid.cloud


### BOB creates DID and a basic msg to Bob

In [173]:
bob_did_to_alice = await create_peer_did(1,1, service_endpoint="https://www.example.com/bob")
print("Bob's DID:", bob_did_to_alice)

Bob's DID: did:peer:2.Ez6LSfqcHmNo54jYA1HoQaa4XmFb9koZSzJf326j1eiFYqRX3.Vz6MknQC3EJevb9sZUQnjvqHxCKoB5G7fP2Re6jgofiGHwPup.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9ib2IiLCJhIjpbImRpZGNvbW0vdjIiXX0


In [174]:

bob_basic_message = Message(
    id = str(uuid.uuid4()),
    type="https://didcomm.org/basicmessage/2.0/message",
    body={"content": "Hola Alice Como estas?"},
    created_time= int(datetime.datetime.now().timestamp())            
)

In [175]:
# bob_response_message = Message(
#     custom_headers = [{
#         "return_route": "all"}],
#     id = str(uuid.uuid4()),
#     #pthid = received_msg_decoded["id"],
#     type = "https://didcomm.org/discover-features/2.0/queries",
#     body = {
#         "queries": [
#             { "feature-type": "protocol", "match": "https://didcomm.org/basicmessage/2.*" }
#         ]
#     }
# )

In [176]:
bob_basic_message_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = bob_response_message,
    frm = bob_did_to_alice,
    to = alice_did,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

In [177]:
print(json.loads(bob_response_message_packed.packed_msg)["protected"])

eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTNRMU0zUXpWekZuZFZZNGJYQkZXR1pGVlZwS1JtTnljREZPZEc5VFNGQTJSbFJFUW5wNk1WbHZkRlZXTGxaNk5rMXJjM0Z5UkhOMlNsZFJVVzVUTkdGRmRGbzNRa1JIWW5wdmMxRm9RMGRuTkdOWFREVnJlbEk1YjJ0V1kyVXVVMlY1U25CYVEwazJTVzAxYkdSNU1YQmFRMGx6U1c1UmFVOXBTbXRpVTBselNXNU5hVTlwU205a1NGSjNZM3B2ZGt3elpETmtlVFZzWlVkR2RHTkhlR3hNYlU1MllsTTVhV0l5U1dsTVEwcG9TV3B3WWtsdFVuQmFSMDUyWWxjd2RtUnFTV2xZV0RBak5reFRkRFV6ZEROWE1XZDFWamh0Y0VWWVprVlZXa3BHWTNKd01VNTBiMU5JVURaR1ZFUkNlbm94V1c5MFZWWSIsImFwdiI6Ijk2UDZQbkVSOXJjYkEwZDBZZldpZVFZbEhMUV84NjRYQVVyVXpPeVBDT0kiLCJza2lkIjoiZGlkOnBlZXI6Mi5FejZMU3Q1M3QzVzFndVY4bXBFWGZFVVpKRmNycDFOdG9TSFA2RlREQnp6MVlvdFVWLlZ6Nk1rc3FyRHN2SldRUW5TNGFFdFo3QkRHYnpvc1FoQ0dnNGNXTDVrelI5b2tWY2UuU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwzZDNkeTVsZUdGdGNHeGxMbU52YlM5aWIySWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAjNkxTdDUzdDNXMWd1VjhtcEVYZkVVWkpG

### Bobs cread DID and msg routed via Mediator

In [178]:
bob_did_to_mediator = await create_peer_did(1,1, service_endpoint="https://www.example.com/bob")

In [179]:
bob_did_to_mediator

'did:peer:2.Ez6LSqQeNL2Mqv7PcdmMy6KZrvMyqLYBYuttcv15TiyiCk5hn.Vz6MkqysyFzYA6hqC23VC5eczrW6UPfAPvQxEFKMkzYf7gWsy.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9ib2IiLCJhIjpbImRpZGNvbW0vdjIiXX0'

In [180]:
bob_routed_message = Message(
    id = str(uuid.uuid4()),
    type="https://didcomm.org/routing/2.0/forward",
    body={"next": alice_did},
    to=[mediator_did],
    attachments=[Attachment(
            data=AttachmentDataJson(json=json.loads(bob_response_message_packed.packed_msg))
        )]
)

In [181]:
bob_routed_message_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = bob_routed_message,
    frm = bob_did_to_mediator,
    to = mediator_did,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending to Mediator

In [182]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, data = bob_routed_message_packed.packed_msg)


In [183]:
resp.content

b'null'

In [312]:
oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
bob_did_for_mediator = await create_peer_did(1,1)
print("Bob's DID:", bob_did_for_mediator)
bob_mediate_grant = Message(
    custom_headers = [{"return_route": "all"}],
    id = str(uuid.uuid4()),
    type = "https://didcomm.org/coordinate-mediation/2.0/mediate-request",
    body = {}
)
bob_mediate_grant_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = bob_mediate_grant,
    frm = bob_did_for_mediator,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
mediator_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
mediator_endpoint = mediator_did_doc["service"][0]["serviceEndpoint"]
print(mediator_endpoint)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, data = bob_mediate_grant_packed.packed_msg)
print(resp.json())

mediator_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(mediator_unpack_msg.message.type)
print("Routing Key: " + mediator_unpack_msg.message.body["routing_did"])
print("Mediator new DID: " + mediator_unpack_msg.message.from_prior.sub)
# mediator rotated did
mediator_routing_key = mediator_unpack_msg.message.body["routing_did"]
bob_mediator_did = mediator_unpack_msg.message.from_prior.sub

#bob_did_new = await create_peer_did(1, 1, service_endpoint=mediator_endpoint, service_routing_keys=[mediator_routing_key])
bob_did_to_alice = await create_peer_did(1, 1, service_endpoint=[{"uri": mediator_routing_key}])

print("bob's DID for Bob:", bob_did_to_alice)

bob_keylist_update = Message(
    id =str(uuid.uuid4()),
    type = "https://didcomm.org/coordinate-mediation/2.0/keylist-update",
    body = {
        "updates":[
        {
            "recipient_did": bob_did_to_alice,
            "action": "add"
        }
    ]
    }
)
bob_keylist_update_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = bob_keylist_update,
    frm = bob_did_for_mediator,
    to = bob_mediator_did,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
mediator_unpack_msg2 = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(mediator_unpack_msg2.message.type)
print(mediator_unpack_msg2.message.body)
print("SEND")
print(bob_did_to_alice)

Bob's DID: did:peer:2.Ez6LSkWW74ZsDXVdix9gnJcopDy8mtq5noTmhidn92inq3AtS.Vz6MkkcNRx5VsDjMmqLACzxRmTQhvAPUrhBniZPHVk8DiNE2r
https://mediator.rootsid.cloud
{"protected":"eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTJKMFNHOXFXVzgxUVcxQ05FSXlkazUxVkd0RFpXRjBTbGhDYVc5MWIycDNPRGRrYjJsV1pWSXhRa04wTGxaNk5rMXJkakk1WW1WU2FUTllhRk0yYWprMFlVUmxVblJWY2pOQ01UZDZha1Z2V0VoM2MzTlpXbk5pY0ZOcGIzVXVVMlY1U25CYVEwazJTVzAxYkdSNU1YQmFRMGx6U1c1UmFVOXBTbXRpVTBselNXNU5hVTlwU205a1NGSjNZM3B2ZGt3eU1XeGFSMnhvWkVjNWVVeHVTblppTTFKNllWZFJkVmt5ZUhaa1YxRnBURU5LYUVscWNHSkpiVkp3V2tkT2RtSlhNSFprYWtscFdGZ3dJelpNVTJKMFNHOXFXVzgxUVcxQ05FSXlkazUxVkd0RFpXRjBTbGhDYVc5MWIycDNPRGRrYjJsV1pWSXhRa04wIiwiYXB2IjoiYS00TnIwZ0wxU19YY0pGQTI2LVhhVDR4eFljcEU1NUJjbEozemVXRlV0TSIsInNraWQiOiJkaWQ6cGVlcjoyLkV6NkxTYnRIb2pZbzVBbUI0QjJ2TnVUa0NlYXRKWEJpb3Vvanc4N2RvaVZlUjFCQ3QuVno2TWt2MjliZVJpM1hoUzZqOTRhRGVSdFVyM0IxN3pqRW9YSHdzc1lac2JwU2lvd

In [327]:
mediator_did_doc

{'id': 'did:peer:2.Ez6LSo7greAQ4GZRuparQC1cysxubcCFjVVZdsvDSocEkL971.Vz6MktiFh7h3jHcHtxoSWCNj1oB5YRsTgoX8SnDo9HBZmdRdx.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0',
 'authentication': [{'id': 'did:peer:2.Ez6LSo7greAQ4GZRuparQC1cysxubcCFjVVZdsvDSocEkL971.Vz6MktiFh7h3jHcHtxoSWCNj1oB5YRsTgoX8SnDo9HBZmdRdx.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0#6MktiFh7h3jHcHtxoSWCNj1oB5YRsTgoX8SnDo9HBZmdRdx',
   'type': 'Ed25519VerificationKey2020',
   'controller': 'did:peer:2.Ez6LSo7greAQ4GZRuparQC1cysxubcCFjVVZdsvDSocEkL971.Vz6MktiFh7h3jHcHtxoSWCNj1oB5YRsTgoX8SnDo9HBZmdRdx.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0',
   'publicKeyMultibase': 'z6MktiFh7h3jHcHtxoSWCNj1oB5YRsTgoX8SnDo9HBZmdRdx'}],
 'keyAgreement': [{'id': 'did:peer:2.Ez6LSo7greAQ4GZRuparQC1cysxubcCFjVVZdsvDSocEkL971.Vz6MktiFh7h3jHcHtxoSWCNj1

In [310]:
#decode base64
import base64,json
peer_url ="https://mediator.rootsid.cloud/qr?_oobid=f3516bf74f92413d91dafcdc1dd988e2"
data=requests.get(peer_url)
data
document=json.loads(base64.b64decode(data.url.split("=",1)[1]))

alice_did_doc = json.loads(peer_did.resolve_peer_did(document['from']))
mediator_did = alice_did_doc["service"][0]["serviceEndpoint"]
print(mediator_did)
mediator_endpoint = mediator_did_doc["service"][0]["serviceEndpoint"]
print(mediator_endpoint)

did:peer:2.Ez6LSszPKZSsfBrvyeSpfTqoSounUmZwCoxoLvijiTXEx3p7G.Vz6MkhdHXej5btPVycSdXsQLb2JBwi6NBYVG7NXRbs1Ro8fJz.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0
https://mediator.rootsid.cloud


In [324]:
bob_did_to_alice = await create_peer_did(1, 1,service_endpoint=[{"uri": mediator_routing_key}])
bob_basic_message = Message(
    id = str(uuid.uuid4()),
    type="https://didcomm.org/basicmessage/2.0/message",
    body={"content": 'Again Alexa', "displayName": 'Alex'},
    created_time= int(datetime.datetime.now().timestamp())            
)

bob_response_message_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = bob_basic_message,
    frm = bob_did_to_alice,
    to = document['from'],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
bob_routed_message = Message(
    id = str(uuid.uuid4()),
    type="https://didcomm.org/routing/2.0/forward",
    body={"next": document['from']},
    to=[mediator_did],
    attachments=[Attachment(
            data=AttachmentDataJson(json=json.loads(bob_response_message_packed.packed_msg))
        )]
)

bob_routed_message_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = bob_routed_message,
    frm = bob_did_to_alice,
    to = mediator_did,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, data = bob_routed_message_packed.packed_msg)


In [326]:
resp.content

b'{"detail":"Malformed Message"}'

In [320]:
json.loads(peer_did.resolve_peer_did(bob_did_to_alice)).keys()

dict_keys(['id', 'authentication', 'keyAgreement', 'service'])

In [321]:
json.loads(peer_did.resolve_peer_did(await create_peer_did(1,1,service_endpoint='https://example.com'))).keys()

dict_keys(['id', 'authentication', 'keyAgreement', 'service'])